In [66]:
import pandas as pd
df = pd.read_csv('./data/gemstone.csv')
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [67]:
df=df.drop(labels=['id'], axis=1)

In [68]:
X = df.drop(labels=['price'], axis=1)
Y=df[['price']]

In [69]:
categorocal_cols = X.select_dtypes(include = 'object').columns
numerical_cols = Y.select_dtypes(include='object').columns

In [85]:
categorocal_cols

Index(['cut', 'color', 'clarity'], dtype='object')

In [70]:
numerical_cols

Index([], dtype='object')

In [71]:
categorocal_cols

Index(['cut', 'color', 'clarity'], dtype='object')

In [72]:
X['cut'].unique()

array(['Premium', 'Very Good', 'Ideal', 'Good', 'Fair'], dtype=object)

In [73]:
cut_categories = ['Premium', 'Very Good', 'Ideal', 'Good', 'Fair']
color_categories = ['F', 'J', 'G', 'E', 'D', 'H', 'I']
clarity_categories = ['VS2', 'SI2', 'VS1', 'SI1', 'IF', 'VVS2', 'VVS1', 'I1']

In [74]:
from sklearn.impute import SimpleImputer ## to handle missing values
from sklearn.preprocessing import StandardScaler ## for nurerial var feature sacling
from sklearn.preprocessing import OrdinalEncoder  ## for cat var ordinal ranking

from sklearn.pipeline import Pipeline   ##Pipeline of transforms with a final estimator
from sklearn.compose import ColumnTransformer   ##transformers to columns of an array or pandas DataFrame

In [75]:
num_pipeline = Pipeline(
    steps = [
    ('imputer', SimpleImputer(strategy = 'median')), ##missing values replaes with median
    ('scaler', StandardScaler())    ##for mean=0 and var=1
    ]

)
cat_pipeline = Pipeline(
    steps = [
    ('imputer',SimpleImputer(strategy='most_frequent')), ##missing values replaes with most freq 
    ('ordinalencoder', OrdinalEncoder(categories=[cut_categories, color_categories, clarity_categories])),
    ('scaler', StandardScaler())

    ]

)


preprocessor = ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('cat_pipeline',cat_pipeline,categorocal_cols)

])

In [76]:
#Train_test_Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.3, random_state=30)

In [78]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train))

In [79]:
X_test = pd.DataFrame(preprocessor.transform(X_test))

In [80]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [81]:
regression = LinearRegression()
regression.fit(X_train, y_train)

LinearRegression()

In [82]:
regression.coef_

array([[-405.68984233,  283.4372593 , -501.43040689]])

In [83]:
regression.intercept_

array([3970.76628955])

In [86]:
import numpy as np
def evaluate_model(true, predicted):
    mae=mean_absolute_error(true, predicted)
    mse=mean_squared_error(true, predicted)
    rmse=np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [107]:
models = {
    'LinearRegression' : LinearRegression(),
    'Lasso': Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet(),
}

trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
     model=list(models.values())[i]
     model.fit(X_train, y_train)

     y_pred = model.predict(X_test)

     mae, rmse, r2_square=evaluate_model(y_test,y_pred)

     print(list(models.keys())[i])

     model_list.append(list(models.keys())[i])

     print('Model Training Performance')
     print("RMSE:",rmse)
     print("MAE:",mae)
     print("R2 score",r2_square*100)

     r2_list.append(r2_square)
    
     print('='*35)
     print('\n')



LinearRegression
Model Training Performance
RMSE: 3966.144032028809
MAE: 2983.0776786130496
R2 score 3.4315664496075593


Lasso
Model Training Performance
RMSE: 3966.154281599689
MAE: 2983.188594589159
R2 score 3.4310673319379204


Ridge
Model Training Performance
RMSE: 3966.144058926794
MAE: 2983.0780312932584
R2 score 3.431565139772985


ElasticNet
Model Training Performance
RMSE: 3974.142085116547
MAE: 3003.769661671075
R2 score 3.041697491330142




In [98]:
models.keys()

dict_keys(['LinearRegression', 'Lasso', 'Ridge', 'ElasticNet'])